# Make a coastal sfincs model from scratch using hydromt

This notebook demonstrates how to prepare and inspect a SFINCS model from scratch. The notebook does not contain the SFINCS code or executables to run the model with. 

We will make a schematization with the following features:
* we only simulate coastal flooding, we do not consider any interactions with riverine flooding
* we use the MERIT Hydro dataset as elevation
* we include boundary conditions from the Global Tide and Surge Model by imposing a number of boundary points where forcing is expected

All lines in this notebook which start with `!` are executed from the command line. Within jupyter (except for jupyter lab) the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### HydroMT CLI build interface


First let us see what hydromt provides us with.

In [ ]:
!hydromt --help

With the `models` flag we can ask if hydroMT recognizes the SFINCS plugin (SFINCS should be mentioned):

In [ ]:
!hydromt --models

It looks like we have several commands that we can consider. We want to `build` a model. How does that work? We can also call `--help` on the available hydroMT commands as follows.

In [ ]:
!hydromt build --help

### Building a first model (and getting an error when data is missing)
Let's try to build an entire schematization, just from the command line (Awesome!!!). In the `--help` section, there is an example for making a model from a bounding box, which is very intuitive. Let's try exactly that. We store the model on the local path `./texel_sfincs`

In [ ]:
!hydromt build sfincs ./texel_sfincs "{'bbox': [4.68,52.97,4.96,53.20]}"

The above might have taken a few seconds, but then should have miserably crashed! What happened:
* HydroMT was looking for a data catalog to use. 
* It did not find any and therefore, it automatically downloaded a sample data catalog which you can now find on your home folder under .hydromt_data
* It tried to build a model in the Texel bounding box, but...alas, the sample dataset does not cover Texel at all! To add datasets covering the area outside the sample data extent, setup your own datasets using data yml files [hydroMT data](https://deltares.github.io/hydromt/latest/user_guide/data.html) using `--data PATH` option, or if your are connected to the Deltares network `--deltares-data` flag.


### Building a first model (and getting it right)
Let's adapt the bounding box to an area that fits within the sample dataset. The dataset covers the Piave basin and its surroundings in Italy. A good place to get a CSV formatted bounding box is https://boundingbox.klokantech.com/

We also add `-vv` for extra verbosity

In [ ]:
!hydromt build sfincs ./venice_sfincs "{'bbox': [12.05,45.30,12.85,45.65]}" -vv

This should work without any errors. 

The example above means the following: run **hydromt build** with:

* `sfincs` : i.e. build a SFINCS model
* `./venice_sfincs` : output model folder
* `"{'bbox': [12.05,45.30,12.85,45.65]}"` : make a model that is active for the full given bounding box. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options)
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

NOTE: As we did not specify a model configuration, only the model region has been setup. To build a complete model we need the use a configuration ini-file.

When we inspect the newly created `./venice_sfincs` folder we only find a log file and the gis folder containing the region geometry saved as geojson file.

In [ ]:
!ls ./venice_sfincs

### Building a complete model (ready to run)

We can provide many more settings in an configuration (.ini) file instead of command line arguments. If you don't provide an ini file (as done above), only the model region is determined. To complete your model you can add more [model components](https://deltares.github.io/hydromt_sfincs/latest/user_guide/sfincs.html) to the model setup.

Let's have a look a typical ini file for setting up coastal SFINCS model, where each `[setup_<>]` refers to a model component. All options which end with `_fn` refers to a data source from the data catalog or a path to data on your machine, see [hydroMT data](https://deltares.github.io/hydromt/latest/user_guide/data.html). In this case some sample data which is downloaded to your machine at `$USER_ROOT/.hydromt_data`

In [ ]:
fn_ini = "sfincs_coastal.ini"
with open(fn_ini, "r") as f:
    txt = f.read()
print(txt)

You can see the following sections:

* `[setup_config]`: arguments which are forwarded to the SFINCS model configuration file sfincs.inp
* `[setup_topobathy]`: basic information required for the model domain grid and elevation. These are the minimal demanded static layers that any SFINCS model needs. Here you may for instance consider altering the elevation and bathymetry data for local data sources. These then have to be supplied in the `data_catalog.yml`.
* `[setup_merge_topobathy]`: add additional data source for bathymetry data to merge with the basemap elevation data.
* `[setup_mask]`: set valid model cells based on the topobathy data and a minimal depth boundary (more options available).
* `[setup_cn_infiltration]`: potential maximum soil retention maps to be used in the curve number method, see [SFINCS docs](https://sfincs.readthedocs.io/en/latest/input.html?highlight=curve#spatially-varying-curve-number) for more information
* `[setup_manning_roughness]`: configuration of the Manning roughness values, typically consisting of a land use or land cover map, and a lookup table that relates land use classes to manning roughness values in sec m$^{-1/3}$
* `[setup_bounds]`: set cells at the model domain edge and within the sea polygon to waterlevel boundary cells.
* `[setup_h_forcing]`: water level forcing data. This can be provided from a set of csv files, see data_sources.yml, but also netcdf files, see [hydroMT docs](https://deltares.github.io/hydromt/latest/user_guide/data.html). 
* `[setup_p_forcing_from_grid]`: gridded precipitation data, that can be used to do compound flood simulations. The `[setup_cn_infiltration]` is then quite essential to include.

To build a model from scratch add the ini file to the command line `hydromt build` method at a 150 m resolution.

In [ ]:
!hydromt build sfincs ./venice_sfincs_150 "{'bbox': [12.05,45.30,12.85,45.65]}" -r 150 -i sfincs_coastal.ini -vv

Let's inspect the output folder. Notice that several SFINCS files have been written.

In [ ]:
!ls ./venice_sfincs_150

One of the files is the SFINCS input file which defines the model, lets have a look:

In [ ]:
fn_inp = "./venice_sfincs_150/sfincs.inp"
with open(fn_inp, "r") as f:
    txt = f.read()

print(txt)

### Inspecting the model
Let's have a closer look at the model. To interact with a model within a notebook or python script, we need to load a number of modules.

In [ ]:
%matplotlib inline
from hydromt_sfincs import SfincsModel

Now we can load the model in memory and inspect it. 

Note that based on the global datasets the lagoon bathymetry is not well represented. A better merge between MERIT hydro topography and GEBCO bathymetry can be achieved by providing an better coastline shapefile (by default OSM based) which together with the merge_method argument determines which dataset is used where.

In [ ]:
# define the model's configuration file
root = r"./venice_sfincs_150"

# read the model with hydromt methods
mod = SfincsModel(root=root, mode="r")
mod.read()
mod.plot_basemap(figsize=(13, 7), shaded=False)

### Next steps
You can update an existing model, e.g. to replace model layers or add forcing. this is described in a follow-up notebook `sfincs_update_model.ipynb`